This notebook shows how to run and compare different reservoirs.

In [1]:
from jacho.layers.reservoirs import RandomReservoir, FastStructuredTransform, SparseReservoir
from jacho.models.generic import GenericEchoState
from jacho.layers.output import Residual
from jacho.training.training import ridge
from jacho.data.KS import KS

from jax import random
import numpy as np
import jax.numpy as jnp

import matplotlib.pyplot as plt

key = random.PRNGKey(42)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
# Setting up our dataset; similar to jonathans
L = 22 / (2 * np.pi)  # length
N = 100  # space discretization step
dt = 0.25  # time discretization step
N_train = 10000
N_test = 2000
N_init = 1000  # remove the initial points
tend = (N_train + N_test) * dt + N_init

np.random.seed(1)
dns = KS(L=L, N=N, dt=dt, tend=tend)
dns.simulate()

In [3]:
# Prepping train and test matrices
# inputs need to be size [time_steps, samples, spatial_points]
# i.e. here they are [10000, 1, 100]
u = np.expand_dims(dns.uu, axis=1)
_, u_train, u_test, _ = np.split(u / np.sqrt(N), 
                                     [N_init, 
                                     (N_init + N_train), 
                                     (N_init + N_train + N_test)], axis=0)

In [8]:
# Setting up random model
n_reservoir = 10000
reservoir_args = (0.4, 0.9, 0.1) #input_scale, reservoir_scale, bias_scale

n_out = u_train.shape[-1]
norm_factor = 1.1 * jnp.sqrt(n_out / n_reservoir)
output_layer_args = (norm_factor, )

model = GenericEchoState(n_reservoir, RandomReservoir, reservoir_args,
                  n_out, Residual, output_layer_args)



state = model.initialize_state(key, n_reservoir)
params = model.init(key, state, u_train[0]) # initializing the parameters and state

In [9]:
%%time
new_state, intermediate_states = model.apply(params, state, u_train, method=model.run_reservoir)

CPU times: user 5.55 s, sys: 4.62 s, total: 10.2 s
Wall time: 10.1 s


In [6]:
# Setting up structured model
n_reservoir = 10000
reservoir_args = (0.4, 0.9, 0.1) #input_scale, reservoir_scale, bias_scale

n_out = u_train.shape[-1]
norm_factor = 1.1 * jnp.sqrt(n_out / n_reservoir)
output_layer_args = (norm_factor, )

model = GenericEchoState(n_reservoir, FastStructuredTransform, reservoir_args,
                  n_out, Residual, output_layer_args)

state = model.initialize_state(key, n_reservoir)
params = model.init(key, state, u_train[0]) # initializing the parameters and state

In [7]:
%%time
new_state, intermediate_states = model.apply(params, state, u_train, method=model.run_reservoir)

CPU times: user 2.3 s, sys: 8.41 ms, total: 2.31 s
Wall time: 1.63 s


In [11]:
# Setting up sparse model model
n_reservoir = 10000
reservoir_args = (0.05, 0.4, 0.9, 0.1) #sparsity level, input_scale, reservoir_scale, bias_scale

n_out = u_train.shape[-1]
norm_factor = 1.1 * jnp.sqrt(n_out / n_reservoir)
output_layer_args = (norm_factor, )

model = GenericEchoState(n_reservoir, SparseReservoir, reservoir_args,
                  n_out, Residual, output_layer_args)

state = model.initialize_state(key, n_reservoir)
params = model.init(key, state, u_train[0]) # initializing the parameters and state

In [12]:
%%time
new_state, intermediate_states = model.apply(params, state, u_train, method=model.run_reservoir)

CPU times: user 1.86 s, sys: 1.24 s, total: 3.1 s
Wall time: 3.05 s
